# An Integer Programming Model for the Sudoku Problem

**Reference**: An Integer Programming Model for the Sudoku Problem by A.C. Bartlett, et. al., The Journal of Online Mathematics and Its Applications 8 (2008).

## 2. Question 1: Solving the Puzzle

Let us mathematically model the Sudoku puzzle as a linear program.

More specifically, we will formulate a binary integer program (BIP) for general $n\times n$ puzzles, where $n=m^2$ for some positive integer.
Once the program is developed to solve the BIP, it can be easily adapted to solve *any* Sudoku puzzle.

### Decision variables

To begin, we define our decision variables:
$$
x_{ijk} = \begin{cases} 1, & \text{if element $(i,j)$ of the $n\times n$ Sudoku matrix contains the integer $k$,} \\ 0, & \text{otherwise.} \end{cases}
$$
When the values of the decision variables are determined, we will know whether each integer $k$ ($1\leq k\leq n$) appears in each element $(i,j)$ of the $n\times n$ Sudoku matrix.
That is, the solution to the corresponding Sudoku puzzle will be defined.

### Objective and constraints

We now turn to the objective function and the set of constraints.

A BIP formulation suitable for Sudoku puzzle is as follows:
\begin{align*}
\text{minimize} &\quad \mathbf{0}^T\mathbf{x} \\
\text{subject to} &\quad \sum_{i=1}^n x_{ijk} = 1 \quad\text{for all $1\leq j,k\leq n$} \tag{1} \\
&\quad \sum_{j=1}^n x_{ijk} = 1 \quad\text{for all $1\leq i,k\leq n$} \tag{2} \\
&\quad \sum_{k=1}^n x_{ijk} = 1 \quad\text{for all $1\leq i,j\leq n$} \tag{3} \\
&\quad \sum_{i=m(p-1)+1}^{mp} \sum_{j=m(q-1)+1}^{mq} x_{ijk} = 1 \quad\text{for all $1\leq k\leq n$ and $1\leq p,q\leq m$} \tag{4} \\
&\quad x_{ijk} = 1 \quad\text{for all $(i,j,k)\in G$} \tag{5} \\
&\quad x_{ijk}\in\{0,1\}
\end{align*}

- **Constraint (1)**: Only one $k$ in each column
- **Constraint (2)**: Only one $k$ in each row
- **Constraint (3)**: Every position in matrix must be filled
- **Constraint (4)**: Only one $k$ in each submatrix
- **Constraint (5)**: All initial elements in $G$ are set "on"

In [1]:
import numpy as np
import cvxpy as cp

def solve_sudoku(puzzle):
    G = np.asarray(puzzle, dtype=np.int64)
    n, _n = G.shape
    assert n == _n
    m = int(np.sqrt(n))
    assert n == m ** 2
    assert np.all((G >= 0) & (G <= n)) # all values are in {0,1,...,n}

    n_ones = np.ones(n)
    n_ones_d2 = np.ones((n, n))

    x = cp.Variable((n, n, n), boolean=True)
    constraints = [
        # (1) only one k in each column
        cp.sum(x, axis=0) == n_ones_d2,
        # (2) only one k in each row
        cp.sum(x, axis=1) == n_ones_d2,
        # (3) every position in matrix must be filled
        cp.sum(x, axis=2) == n_ones_d2,
    ] + [
        # (4) only one k in each submatrix
        cp.sum(
            x[m*p:m*(p+1), m*q:m*(q+1), :], axis=(0, 1)
        ) == n_ones for p in range(m) for q in range(m)
    ] + [
        # (5) All initial elements in G are set "on"
        x[i, j, G[i, j]-1] == 1 for i, j in np.argwhere(G)
    ]

    prob = cp.Problem(cp.Minimize(0), constraints)
    prob.solve(canon_backend="SCIPY") # for problems with expressions of dim > 2

    if prob.status == "optimal":
        return (np.where(x.value)[2] + 1).reshape((n, n))
    else: # infeasible or unbounded
        raise RuntimeError("Failed to find a solution.")

In [2]:
puzzle = [
    [0, 2, 0, 0],
    [3, 0, 0, 0],
    [0, 0, 4, 3],
    [0, 3, 0, 0]
]
solve_sudoku(puzzle)

array([[1, 2, 3, 4],
       [3, 4, 1, 2],
       [2, 1, 4, 3],
       [4, 3, 2, 1]])

![](figures/sudoku_figure1.png)

In [3]:
puzzle = [
    [0, 0, 0, 0, 0, 0, 0, 2, 0],
    [0, 2, 0, 0, 0, 0, 5, 0, 0],
    [0, 0, 7, 0, 0, 3, 4, 0, 0],
    [2, 0, 0, 1, 0, 0, 3, 4, 0],
    [6, 4, 0, 0, 8, 0, 0, 5, 9],
    [0, 9, 5, 0, 0, 2, 0, 0, 1],
    [0, 0, 3, 4, 0, 0, 8, 0, 0],
    [0, 0, 9, 0, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0]
]

![](figures/sudoku_figure2.png)

In [4]:
solve_sudoku(puzzle)

array([[9, 3, 4, 5, 6, 8, 1, 2, 7],
       [8, 2, 6, 7, 1, 4, 5, 9, 3],
       [1, 5, 7, 9, 2, 3, 4, 6, 8],
       [2, 7, 8, 1, 5, 9, 3, 4, 6],
       [6, 4, 1, 3, 8, 7, 2, 5, 9],
       [3, 9, 5, 6, 4, 2, 7, 8, 1],
       [5, 6, 3, 4, 9, 1, 8, 7, 2],
       [7, 8, 9, 2, 3, 5, 6, 1, 4],
       [4, 1, 2, 8, 7, 6, 9, 3, 5]])

# Sudoku.com

## What is Extreme Sudoku?

Extreme Sudoku is the most challenging version of the 9x9 grid puzzle game for the real puzzle masters! It's not just about following the basic rules - you've got to be a puzzle-solving expert and use advanced strategies to crack it. 

Extreme Sudoku level provides players with very few given digits and requires them to push their mental limits and test strategic thinking like never before. If you're an experienced Sudoku enthusiast looking for a challenge, Extreme Sudoku is your ticket to a thrilling adventure.

![](figures/sudoku_extreme.png)

In [5]:
extreme_puzzle = [
    [5, 0, 0, 0, 8, 2, 3, 0, 4],
    [0, 7, 0, 0, 0, 0, 0, 0, 5],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [7, 0, 0, 0, 3, 5, 2, 0, 0],
    [0, 0, 0, 4, 0, 0, 0, 0, 0],
    [0, 0, 9, 0, 0, 0, 0, 6, 0],
    [0, 6, 0, 0, 4, 8, 0, 2, 0],
    [8, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 7, 0, 0, 4, 0, 0]
]
solve_sudoku(extreme_puzzle)

array([[5, 9, 1, 6, 8, 2, 3, 7, 4],
       [6, 7, 2, 3, 9, 4, 8, 1, 5],
       [3, 8, 4, 5, 7, 1, 6, 9, 2],
       [7, 1, 6, 8, 3, 5, 2, 4, 9],
       [2, 5, 8, 4, 6, 9, 1, 3, 7],
       [4, 3, 9, 2, 1, 7, 5, 6, 8],
       [1, 6, 5, 9, 4, 8, 7, 2, 3],
       [8, 4, 7, 1, 2, 3, 9, 5, 6],
       [9, 2, 3, 7, 5, 6, 4, 8, 1]])

## 3. Solving Variations on Sudoku Puzzles

### 3.1 Sudoku X

The "Sudoku X" puzzle is like the standard puzzle, with an extra requirement: The two long diagonals of the board must also contain each digit from 1 to 9 exactly once.

Thus, any solution to the Sudoku X puzzle is also a solution to the standard Sudoku puzzle, but the converse is not the case.

![](figures/sudoku_figure3.png)

To capture the requirement for the positive diagonal, we add the nine constraints
$$
\sum_{r=1}^9 x_{rrk} = 1 \quad\text{for $1\leq k\leq 9$}.
$$
Similarly, for the anti-diagonal, the following set of nine constraints are added
$$
\sum_{r=1}^9 x_{r(10-r)k} = 1 \quad\text{for $1\leq k\leq 9$}.
$$

In [6]:
import numpy as np
import cvxpy as cp

def solve_sudoku_x(puzzle):
    G = np.asarray(puzzle, dtype=np.int64)
    m = 3
    n = m ** 2
    assert G.shape == (n, n)
    assert np.all((G >= 0) & (G <= n)) # all values are in {0,1,...,n}

    n_ones = np.ones(n)
    n_ones_d2 = np.ones((n, n))

    x = cp.Variable((n, n, n), boolean=True)
    constraints = [
        # (1) only one k in each column
        cp.sum(x, axis=0) == n_ones_d2,
        # (2) only one k in each row
        cp.sum(x, axis=1) == n_ones_d2,
        # (3) every position in matrix must be filled
        cp.sum(x, axis=2) == n_ones_d2,
    ] + [
        # (4) only one k in each submatrix
        cp.sum(
            x[m*p:m*(p+1), m*q:m*(q+1), :], axis=(0, 1)
        ) == n_ones for p in range(m) for q in range(m)
    ] + [
        # (5) All initial elements in G are set "on"
        x[i, j, G[i, j]-1] == 1 for i, j in np.argwhere(G)
    ] + [
        # diagonal
        cp.sum(cp.diag(x[:, :, k])) == 1 for k in range(n)
    ] + [
        # anti-diagonal (numpy.fliplr)
        cp.sum(cp.diag(x[:, ::-1, k])) == 1 for k in range(n)
    ]

    prob = cp.Problem(cp.Minimize(0), constraints)
    prob.solve(canon_backend="SCIPY") # for problems with expressions of dim > 2

    if prob.status == "optimal":
        return (np.where(x.value)[2] + 1).reshape((n, n))
    else: # infeasible or unbounded
        raise RuntimeError("Failed to find a solution.")

In [7]:
puzzle = [
    [8, 0, 0, 0, 0, 0, 0, 0, 2],
    [4, 0, 0, 0, 0, 0, 0, 0, 7],
    [0, 7, 0, 0, 0, 0, 0, 9, 0],
    [0, 0, 5, 0, 0, 0, 4, 0, 0],
    [0, 0, 0, 1, 4, 5, 0, 0, 0],
    [0, 0, 6, 0, 0, 0, 9, 0, 0],
    [0, 3, 0, 0, 0, 0, 0, 8, 0],
    [9, 0, 0, 0, 0, 0, 0, 0, 4],
    [1, 0, 0, 0, 0, 0, 0, 0, 6]
]
solve_sudoku_x(puzzle)

array([[8, 5, 1, 9, 7, 3, 6, 4, 2],
       [4, 9, 2, 6, 5, 1, 8, 3, 7],
       [6, 7, 3, 2, 8, 4, 5, 9, 1],
       [3, 1, 5, 7, 6, 9, 4, 2, 8],
       [2, 8, 9, 1, 4, 5, 7, 6, 3],
       [7, 4, 6, 8, 3, 2, 9, 1, 5],
       [5, 3, 7, 4, 2, 6, 1, 8, 9],
       [9, 6, 8, 3, 1, 7, 2, 5, 4],
       [1, 2, 4, 5, 9, 8, 3, 7, 6]])

### 3.2 Four Square Sudoku

The "Four Square" Sudoku is again a standard puzzle but with an extra requirement.

There are four shaded $3\times 3$ regions on the Sudoku board, and in addition to the requirements of the standard Sudoku, each shaded region must also contain each digit from 1 to 9 exactly once. 

![](figures/sudoku_figure4.png)

In addition to the standard Sudoku constraints, the Four Square Sudoku puzzle requires that each shaded $3\times3$ square contains each digit exactly once.

This requirement results in 36 constraints on top of the standard constraints (4 squares, 9 digits). The constraints themselves look similar to the other constraints on the $3\times3$ subgrids:
$$
\sum_{r=i}^{i+2} \sum_{c=j}^{j+2} x_{rck} = 1 \quad\text{for $i,j\in\{2,6\}$ and $1\leq k\leq 9$}.
$$

In [8]:
import numpy as np
import cvxpy as cp

def solve_four_square_sudoku(puzzle):
    G = np.asarray(puzzle, dtype=np.int64)
    m = 3
    n = m ** 2
    assert G.shape == (n, n)
    assert np.all((G >= 0) & (G <= n)) # all values are in {0,1,...,n}

    n_ones = np.ones(n)
    n_ones_d2 = np.ones((n, n))

    x = cp.Variable((n, n, n), boolean=True)
    constraints = [
        # (1) only one k in each column
        cp.sum(x, axis=0) == n_ones_d2,
        # (2) only one k in each row
        cp.sum(x, axis=1) == n_ones_d2,
        # (3) every position in matrix must be filled
        cp.sum(x, axis=2) == n_ones_d2,
    ] + [
        # (4) only one k in each submatrix
        cp.sum(
            x[m*p:m*(p+1), m*q:m*(q+1), :], axis=(0, 1)
        ) == n_ones for p in range(m) for q in range(m)
    ] + [
        # (5) All initial elements in G are set "on"
        x[i, j, G[i, j]-1] == 1 for i, j in np.argwhere(G)
    ] + [
        # four squares
        cp.sum(
            x[i:i+3, j:j+3, :], axis=(0, 1)
        ) == n_ones for i in [1, 5] for j in [1, 5]
    ]

    prob = cp.Problem(cp.Minimize(0), constraints)
    prob.solve(canon_backend="SCIPY") # for problems with expressions of dim > 2

    if prob.status == "optimal":
        return (np.where(x.value)[2] + 1).reshape((n, n))
    else: # infeasible or unbounded
        raise RuntimeError("Failed to find a solution.")

In [9]:
puzzle = [
    [0, 0, 7, 0, 0, 4, 0, 0, 1],
    [0, 0, 0, 2, 8, 0, 0, 0, 0],
    [2, 0, 6, 0, 0, 9, 0, 0, 0],
    [0, 5, 0, 0, 0, 0, 2, 0, 6],
    [0, 1, 0, 0, 2, 0, 0, 9, 0],
    [6, 0, 4, 0, 0, 0, 0, 7, 0],
    [0, 0, 0, 8, 0, 0, 9, 0, 2],
    [0, 0, 0, 0, 7, 2, 0, 0, 0],
    [8, 0, 0, 4, 0, 0, 6, 0, 0]
]
solve_four_square_sudoku(puzzle)

array([[9, 3, 7, 5, 6, 4, 8, 2, 1],
       [5, 4, 1, 2, 8, 3, 7, 6, 9],
       [2, 8, 6, 7, 1, 9, 4, 5, 3],
       [7, 5, 9, 3, 4, 8, 2, 1, 6],
       [3, 1, 8, 6, 2, 7, 5, 9, 4],
       [6, 2, 4, 1, 9, 5, 3, 7, 8],
       [1, 7, 5, 8, 3, 6, 9, 4, 2],
       [4, 6, 3, 9, 7, 2, 1, 8, 5],
       [8, 9, 2, 4, 5, 1, 6, 3, 7]])

### 3.3 Four Pyramids Sudoku

Like the Four Square variant, the "Four Pyramid" Sudoku is a standard puzzle with four shaded resions, where one must ensure that each shaded region contains exactly one of each digit from 1 to 9 (in addition to satisfying the requirements of the standard Sudoku).

![](figures/sudoku_figure5.png)

Like the previous variant, the Four Pyramids variant adds 36 additional constraints to the standard ones.
Each of the four pyramid-shaped shaded regions must contain each digit exactly once:
\begin{align*}
\sum_{r=1}^3 \sum_{c=3+r}^{9-r} x_{rck} &= 1 \quad\text{for $1\leq k\leq 9$}, \\
\sum_{c=1}^3 \sum_{r=1+c}^{7-c} x_{rck} &= 1 \quad\text{for $1\leq k\leq 9$}, \\
\sum_{r=7}^9 \sum_{c=11-r}^{r-3} x_{rck} &= 1 \quad\text{for $1\leq k\leq 9$}, \\
\sum_{c=7}^9 \sum_{r=13-c}^{c-1} x_{rck} &= 1 \quad\text{for $1\leq k\leq 9$}.
\end{align*}

In [10]:
import numpy as np
import cvxpy as cp

def solve_four_pyramid_sudoku(puzzle):
    G = np.asarray(puzzle, dtype=np.int64)
    m = 3
    n = m ** 2
    assert G.shape == (n, n)
    assert np.all((G >= 0) & (G <= n)) # all values are in {0,1,...,n}

    n_ones = np.ones(n)
    n_ones_d2 = np.ones((n, n))

    x = cp.Variable((n, n, n), boolean=True)
    constraints = [
        # (1) only one k in each column
        cp.sum(x, axis=0) == n_ones_d2,
        # (2) only one k in each row
        cp.sum(x, axis=1) == n_ones_d2,
        # (3) every position in matrix must be filled
        cp.sum(x, axis=2) == n_ones_d2,
    ] + [
        # (4) only one k in each submatrix
        cp.sum(
            x[m*p:m*(p+1), m*q:m*(q+1), :], axis=(0, 1)
        ) == n_ones for p in range(m) for q in range(m)
    ] + [
        # (5) All initial elements in G are set "on"
        x[i, j, G[i, j]-1] == 1 for i, j in np.argwhere(G)
    ] + [
        # four pyramids
        x[0,3,:] + x[0,4,:] + x[0,5,:] + x[0,6,:] + x[0,7,:] + \
        x[1,4,:] + x[1,5,:] + x[1,6,:] + x[2,5,:] == n_ones,
        x[1,0,:] + x[2,0,:] + x[3,0,:] + x[4,0,:] + x[5,0,:] + \
        x[2,1,:] + x[3,1,:] + x[4,1,:] + x[3,2,:] == n_ones,
        x[6,3,:] + x[7,2,:] + x[7,3,:] + x[7,4,:] + x[8,1,:] + \
        x[8,2,:] + x[8,3,:] + x[8,4,:] + x[8,5,:] == n_ones,
        x[3,8,:] + x[4,7,:] + x[4,8,:] + x[5,6,:] + x[5,7,:] + \
        x[5,8,:] + x[6,7,:] + x[6,8,:] + x[7,8,:] == n_ones
    ]

    prob = cp.Problem(cp.Minimize(0), constraints)
    prob.solve(canon_backend="SCIPY") # for problems with expressions of dim > 2

    if prob.status == "optimal":
        return (np.where(x.value)[2] + 1).reshape((n, n))
    else: # infeasible or unbounded
        raise RuntimeError("Failed to find a solution.")

In [11]:
puzzle = [
    [2, 0, 0, 0, 5, 0, 0, 0, 7],
    [0, 7, 5, 6, 0, 0, 0, 3, 0],
    [0, 0, 3, 0, 0, 0, 5, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [5, 0, 0, 0, 7, 0, 0, 0, 2],
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 4, 0, 0, 0, 6, 0, 0],
    [0, 6, 0, 0, 0, 1, 8, 7, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 4]
]
solve_four_pyramid_sudoku(puzzle)

array([[2, 4, 8, 9, 5, 3, 1, 6, 7],
       [1, 7, 5, 6, 2, 8, 4, 3, 9],
       [6, 9, 3, 4, 1, 7, 5, 2, 8],
       [4, 2, 7, 3, 8, 5, 9, 1, 6],
       [5, 8, 9, 1, 7, 6, 3, 4, 2],
       [3, 1, 6, 2, 9, 4, 7, 8, 5],
       [7, 5, 4, 8, 3, 2, 6, 9, 1],
       [9, 6, 2, 5, 4, 1, 8, 7, 3],
       [8, 3, 1, 7, 6, 9, 2, 5, 4]])